##### Quiz) 위키피디아에서 제공되는 지진 데이터를 활용한 Q&A 챗봇을 만드시오.

##### 조건)
1. 거대 언어 모델(llm) : gpt-4o-mini
1. 임베딩 모델 : BAAI/bge-m3
1. 문서 : 위키피디아에서 제공하는 지진 내용 전체 (WikipediaLoader 사용, 한국어, doc_content_chars_max 40000)
1. 텍스트 분리(splitter) : CharacterTextSplitter 사용 (chunk_size=1000, chunk_overlap=100)
1. 벡터DB : Chroma
1. 검색기(retriever) : 기본값으로 하되 최대 검색 청크 수 k=3 으로 설정
1. 프롬프트 템플릿 : langchain-ai/retrieval-qa-chat 활용
1. 출력 파서 : StrOutputParser

##### 질문)
1. 지진파의 종류에 대해서 알려줘
1. 지금까지 있었던 지진 중 가장 규모가 큰 것은?
1. 지진을 주제로 한 영화는 어떤 게 있어?

##### 답변) 아래 내용이 포함되면 정답으로 간주
1. P파, S파, 표면파
1. 칠레 발디비아 지진
1. 대지진, 일본 침몰, 2012

In [ ]:
# 거대 언어 모델(llm) : llama3.1:8b
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1:8b")

In [5]:
!ollama serve

Error: listen tcp 127.0.0.1:11435: bind: Only one usage of each socket address (protocol/network address/port) is normally permitted.


In [6]:
# 2. 임베딩 모델 : BAAI/bge-m3
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "BAAI/bge-m3"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# 3. 문서 : 위키피디아에서 제공하는 지진 내용 전체 (WikipediaLoader 사용, 한국어, doc_content_chars_max 40000)
from langchain_community.document_loaders import WikipediaLoader
loader = WikipediaLoader(query="지진", lang="ko", load_max_docs=1, doc_content_chars_max=40000)

In [8]:
# 4. 텍스트 분리(splitter) : CharacterTextSplitter 사용 (chunk_size=1000, chunk_overlap=100)
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

docs = loader.load_and_split(text_splitter=text_splitter)

Created a chunk of size 1268, which is longer than the specified 1000
Created a chunk of size 1006, which is longer than the specified 1000
Created a chunk of size 1451, which is longer than the specified 1000
Created a chunk of size 2325, which is longer than the specified 1000
Created a chunk of size 1052, which is longer than the specified 1000
Created a chunk of size 1213, which is longer than the specified 1000
Created a chunk of size 1260, which is longer than the specified 1000
Created a chunk of size 1317, which is longer than the specified 1000
Created a chunk of size 1082, which is longer than the specified 1000
Created a chunk of size 1998, which is longer than the specified 1000
Created a chunk of size 1141, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000


In [9]:
# 5. 벡터DB : Chroma
from langchain_chroma import Chroma
db = Chroma.from_documents(docs, hf_embeddings)

In [10]:
# 6. 검색기(retriever) : 기본값으로 하되 최대 검색 청크 수 k=3 으로 설정
retriever = db.as_retriever(search_kwargs={"k": 3})

In [11]:
# 7. 프롬프트 템플릿 : langchain-ai/retrieval-qa-chat 활용
from langchain import hub
prompt_template = hub.pull("langchain-ai/retrieval-qa-chat")

c:\Python311\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
# 8. 출력 파서 : StrOutputParser
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [13]:
# Chain
from langchain_core.runnables import RunnablePassthrough

rag_chain = {
    "context": retriever,
    "input": RunnablePassthrough()
} | prompt_template | llm | parser

In [ ]:
# 결과 확인

# 질문)
# 1. 지진파의 종류에 대해서 알려줘
# 2. 지금까지 있었던 지진 중 가장 규모가 큰 것은?
# 3. 지진을 주제로 한 영화는 어떤 게 있어?

# 답변)
# 1. P파, S파, 표면파
# 2. 칠레 발디비아 지진
# 3. 대지진, 일본 침몰, 2012

In [14]:
rag_chain.invoke("지진파의 종류에 대해서 알려줘")

'지진파는 다양한 종류가 있습니다. 대표적인 지진파의 종류는 다음과 같습니다.\n\n1. **P파 (P-wave)**: 종방향으로 이동하는 충격파 또는 압축파입니다. 지구 내부에서 가장 빠르게 이동하는 파이다.\n2. **S파 (S-wave)**: 횡방향으로 이동하는 전단파입니다. 지구 내부에서 P파보다 느리게 이동하지만, 지표면까지 도달한 후에는 지진 피해를 주는 파다.\n3. **표면파**: 지구 표면 위에서 발생하는 파이다. 레일리파와 러브파가 대표적인 표면파입니다.\n\n이러한 지진파들은 서로 다른 속도로 암반을 통과하며, 지진의 진원 및 지구 내부 구조를 이해하는 데 도움을 줄 수 있습니다.'

In [15]:
rag_chain.invoke("지금까지 있었던 지진 중 가장 규모가 큰 것은?")

'1960년 칠레 발디비아 지진이 지구 역사상 규모가 가장 큰 지진으로 기록되어있다.'

In [16]:
rag_chain.invoke("지진을 주제로 한 영화는 어떤 게 있어?")

'\'지진(地震, 영어: earthquake, quake, tremor, temblor)\'이라는 키워드를 가진 문서의 내용 중에서 "문화에서의 지진" 부분에 나와있는 정보를 보면 다음과 같은 영화가 있는 것 같습니다.\n\n- 《대지진》(1974년) : 1971년 샌페르난도 지진에서 영감을 받았습니다.\n- 《일본 침몰》: 고마쓰 사쿄가 만들었습니다.\n- 《2012》(2009년) : 롤란트 에머리히가 만든 영화입니다.'